# Mojo 0.6
This is a test notebook that explores some of the new features in [Mojo 0.6 release](https://docs.modular.com/mojo/changelog.html?utm_content=273987722&utm_medium=social&utm_source=twitter&hss_channel=tw-1483918307484848132)

In [16]:
@value
struct test: 
  fn sayhello(self):
    print("Hello")

let t = test()
t.sayhello()

Hello


## Movable, Copyable

New traits added for move and copy constructors that are required for types that can be stored in a DynamicVector.

In [17]:

struct van(Movable, Copyable, CollectionElement):
  var thing: Int

  fn __init__(inout self):
    print("van init")
    self.thing = 0

  fn __moveinit__(inout self: van, owned existing: van ):
    print("van moveinit")
    self.thing = existing.thing

  fn __copyinit__(inout self: van, existing: van ):
    print("van copyinit")
    self.thing = existing.thing

  fn __del__(owned self):
    print("del")

  
fn testMovableCopyable():
   print("testMovableCopyable")
   var vans: DynamicVector[van] = DynamicVector[van]() 
   vans.append(van())
   print("len vans",len(vans))
   print("Done")
   # surprising del before
   # explict calling del on vans weird also

## Partial automatic parametarization

Readme says: "when a function is declared with an argument of a partially bound type, the unbound parameters of that type are implicitly added to the function’s input parameters."

This assumes knowledge of what a partially bound type is.  What is that?

From the documentation: [type bindings](https://docs.modular.com/mojo/manual/parameters/#fully-bound-partially-bound-and-unbound-types)

In [19]:
struct diary[storetype: CollectionElement]:
   var storage: DynamicVector[storetype]

   fn __init__(inout self):
      self.storage = DynamicVector[storetype]()

# when we use the diary type, we supply / bind a parameter
var recorder: diary[String] = diary[String]()

# function has a unbound form of diary as function argument
fn partiallyboundfunction(thediary: diary):
   pass

# calling funciton performas the parameter binding
partiallyboundfunction(diary[String]())

# calling funciton performas the parameter binding
partiallyboundfunction(diary[String]())